## Import CPS Data from NBER Website
The CPS data is preprocessed to calculate alternative unemployment rates by the National Bureau of Economic Research (NBER). It is stored in the form:
    https://data.nber.org/cps-basic3/csv/<CCYY>/cpsb<CCYY><MM>.csv
This file downlaods those data and then cleans and imports it into the database.

In [1]:
%load_ext autoreload
%autoreload 2
from etl_eta import load_data
from etl_cps import clean_cps
import pathlib

In [ ]:
DB = pathlib.Path(__file__).parent / "ui_stats.db"
print(DB)

In [2]:
# Create a to-do list to track values
to_do_list = []
for year in range(2000, 2024):
    for month in range(1, 13):
        to_do_list.append(f"{year}-{month}")
        pass

### Make requests to NBER server and iteratively load data

In [5]:
# Testing
df = clean_cps("https://data.nber.org/cps-basic3/csv/2022/cpsb202201.csv")

IncompleteRead: IncompleteRead(137322019 bytes read, 3887423 more expected)

In [ ]:
# Loop through each year
for year in range(2000, 2024):
    for month in range(1, 13):
        if f"{year}-{month}" not in to_do_list:
            continue
        try: 
            print(f"Starting {year}-{month}")
            df = clean_cps(f"https://data.nber.org/cps-basic3/csv/{year}/cpsb{year}{month}.csv")
            print("    Downloading... Done")
            load_data(DB, df, "unemp")
            print("    Posting...done")
            to_do_list.remove(f"{year}-{month}")
        except Exception as e:
            print(f"   Failed as {e}")